# CareHarmony Case Study

You will be using the CMS 2008-2010 Data Entrepreneurs Synthetic PUF beneficiary data files for this project. This dataset is a bit old, but it is a large synthetic dataset that closely mirrors the type of data that we work with. Due to file size limitations, each data type in the CMS Linkable 2008-2010 Medicare DE-SynPUF is released in 20 separate samples (essentially each is a .25% sample)

The DE-SynPUF contains five types of data for the period 2008-2010:
* Beneficiary Summary
* Inpatient Claims
* Outpatient Claims
* Carrier Claims (not included in the current analysis)
* Prescription Drug Events (not included in the current analysis)


To keep things simple, you will be primarily utilizing the beneficiary-level files that can be found in the zip file attached. These data files contain high-level summaries of patient demographics, conditions, and spending for each of the years 2008, 2009, and 2010. It is organized into twenty separate files for each year and organized such that the same patients are retained in each of the yearly files sharing the same index number. Ten files per year are included for completeness, but you may use as many as you like to conduct your analysis. Note that the dataset is indeed synthetic so many relationships we would expect to see in real world data may not hold. Accordingly, in our evaluation we will focus more on your analytical approach than the actual results.

The original source of the files can be found at this link: https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/DESample01. Translations for a lot of the common terms provided in the Beneficiary Summary file can be found at this link: https://www.cms.gov/files/document/de-10-codebook.pdf-0.

# CareHarmony Case Study: predictive modeling

The health system is interested in identifying and prioritizing patients who are at risk of experiencing a spike in costs in 2011. Use the beneficiary data from 2008-2010 to develop a model to address this concern and explain how you would use it to select a set of patients to prioritize in 2011. Please also make sure to explain your model selection and validation procedure.

In [51]:
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from xgboost import plot_importance

from matplotlib import pyplot as plt

Beneficiary data processing

In [52]:
start_time = time.time()

# Define the base directory for our CSV files
base_dir = r'C:\code_personal_use\cms\data\bene'

# List of years we have data for
years = [2008, 2009, 2010]

# Columns to use for each year
columns_2008 = [
    'DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_SEX_IDENT_CD', 'BENE_RACE_CD',
    'SP_STATE_CODE', 'BENE_COUNTY_CD', 'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN',
    'SP_CNCR', 'BENE_ESRD_IND', 'SP_COPD', 'SP_DEPRESSN', 'SP_DIABETES',
    'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA', 'MEDREIMB_IP',
    'BENRES_IP', 'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP', 'PPPYMT_OP',
    'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'
]

columns_2009_2010 = [
    'DESYNPUF_ID', 'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR',
    'BENE_ESRD_IND', 'SP_COPD', 'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT',
    'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA', 'MEDREIMB_IP', 'BENRES_IP',
    'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP', 'PPPYMT_OP', 'MEDREIMB_CAR',
    'BENRES_CAR', 'PPPYMT_CAR'
]

# Function to recode BENE_ESRD_IND
def recode_bene_esrd_ind(df, column):
    df[column] = df[column].replace({'Y': 1, '0': 2}).astype(int)
    return df

# Function to rename columns with a suffix
def rename_columns_with_suffix(df, suffix):
    df = df.rename(columns=lambda x: x + suffix if x != 'DESYNPUF_ID' else x)
    return df

# Function to load and preprocess a batch of files for a given year
def load_and_preprocess_batch(file_paths, year):
    columns = columns_2008 if year == 2008 else columns_2009_2010
    parse_dates = ['BENE_BIRTH_DT'] if year == 2008 else None
    date_parser = lambda x: pd.to_datetime(x, format='%Y%m%d') if year == 2008 else None

    # Load and preprocess files in the batch
    dfs = []
    for file_path in file_paths:
        df = pd.read_csv(
            file_path,
            usecols=columns,
            parse_dates=parse_dates,
            date_parser=date_parser
        )
        df = recode_bene_esrd_ind(df, 'BENE_ESRD_IND')
        df = rename_columns_with_suffix(df, f'_{year}')
        dfs.append(df)

    # Concatenate all DataFrames in the batch
    return pd.concat(dfs, ignore_index=True)

# Create a dictionary to store DataFrames for each year
dfs_by_year = {}

# Process files in batches
batch_size = 5  # Adjust the batch size based on your system's memory
for year in years:
    file_paths = [os.path.join(base_dir, f'DE1_0_{year}_Beneficiary_Summary_File_Sample_{i}.csv') for i in range(1, 20)]
    batches = [file_paths[i:i + batch_size] for i in range(0, len(file_paths), batch_size)]

    # Load and preprocess each batch
    year_dfs = []
    for batch in batches:
        batch_df = load_and_preprocess_batch(batch, year)
        year_dfs.append(batch_df)

    # Concatenate all batches for the year
    dfs_by_year[year] = pd.concat(year_dfs, ignore_index=True)

# Merge DataFrames for all years on DESYNPUF_ID
merged_df = dfs_by_year[2008]
for year in years[1:]:
    merged_df = merged_df.merge(dfs_by_year[year], on='DESYNPUF_ID', how='inner')

# Define the prefixes to check for null values
prefixes = [
    'MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP',
    'PPPYMT_OP', 'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'
]

# Function to replace null values with 0 for specified prefixes
def replace_nulls_with_zero(df, prefixes):
    for prefix in prefixes:
        cols = [col for col in df.columns if col.startswith(prefix)]
        df[cols] = df[cols].fillna(0)
    return df

merged_df = replace_nulls_with_zero(merged_df, prefixes)

# Calculate Age
def calculate_age(birth_date, reference_date):
    return reference_date.year - birth_date.year - ((reference_date.month, reference_date.day) < (birth_date.month, birth_date.day))

reference_date = datetime.now()  # Using the current date as the reference date
merged_df['Age'] = merged_df['BENE_BIRTH_DT_2008'].apply(lambda x: calculate_age(x, reference_date))

# Display the first few rows of the resulting DataFrame and unique ID counts
print(f'Unique IDs in 2008: {dfs_by_year[2008]["DESYNPUF_ID"].nunique()}')
print(f'Unique IDs in 2009: {dfs_by_year[2009]["DESYNPUF_ID"].nunique()}')
print(f'Unique IDs in 2010: {dfs_by_year[2010]["DESYNPUF_ID"].nunique()}')
print(f'Unique IDs after merging: {merged_df["DESYNPUF_ID"].nunique()}')

print("First few rows of the merged DataFrame:")
print(merged_df.head())

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

Unique IDs in 2008: 2210481
Unique IDs in 2009: 2176679
Unique IDs in 2010: 2142287
Unique IDs after merging: 2142287
First few rows of the merged DataFrame:
        DESYNPUF_ID BENE_BIRTH_DT_2008  BENE_SEX_IDENT_CD_2008  \
0  00013D2EFD8E45D1         1923-05-01                       1   
1  00016F745862898F         1943-01-01                       1   
2  0001FDD721E223DC         1936-09-01                       2   
3  00021CA6FF03E670         1941-06-01                       1   
4  00024B3D2352D2D0         1936-08-01                       1   

   BENE_RACE_CD_2008  BENE_ESRD_IND_2008  SP_STATE_CODE_2008  \
0                  1                   2                  26   
1                  1                   2                  39   
2                  1                   2                  39   
3                  5                   2                   6   
4                  1                   2                  52   

   BENE_COUNTY_CD_2008  SP_ALZHDMTA_2008  SP_CHF_2008  SP_CH

In [53]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2142287 entries, 0 to 2142286
Data columns (total 70 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   DESYNPUF_ID             object        
 1   BENE_BIRTH_DT_2008      datetime64[ns]
 2   BENE_SEX_IDENT_CD_2008  int64         
 3   BENE_RACE_CD_2008       int64         
 4   BENE_ESRD_IND_2008      int32         
 5   SP_STATE_CODE_2008      int64         
 6   BENE_COUNTY_CD_2008     int64         
 7   SP_ALZHDMTA_2008        int64         
 8   SP_CHF_2008             int64         
 9   SP_CHRNKIDN_2008        int64         
 10  SP_CNCR_2008            int64         
 11  SP_COPD_2008            int64         
 12  SP_DEPRESSN_2008        int64         
 13  SP_DIABETES_2008        int64         
 14  SP_ISCHMCHT_2008        int64         
 15  SP_OSTEOPRS_2008        int64         
 16  SP_RA_OA_2008           int64         
 17  SP_STRKETIA_2008        int64         
 18  ME

In [54]:
merged_df.describe(())

,BENE_SEX_IDENT_CD_2008,BENE_RACE_CD_2008,BENE_ESRD_IND_2008,SP_STATE_CODE_2008,BENE_COUNTY_CD_2008,SP_ALZHDMTA_2008,SP_CHF_2008,SP_CHRNKIDN_2008,SP_CNCR_2008,SP_COPD_2008,...,MEDREIMB_IP_2010,BENRES_IP_2010,PPPYMT_IP_2010,MEDREIMB_OP_2010,BENRES_OP_2010,PPPYMT_OP_2010,MEDREIMB_CAR_2010,BENRES_CAR_2010,PPPYMT_CAR_2010,Age
count,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,...,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06,2.142287e+06
mean,1.555706e+00,1.284521e+00,1.929264e+00,2.572132e+01,3.655592e+02,1.807819e+00,1.715481e+00,1.839042e+00,1.936368e+00,1.864491e+00,...,1.244053e+03,1.448212e+02,5.237660e+01,4.336826e+02,1.314106e+02,1.503832e+01,8.477981e+02,2.394512e+02,1.284738e+01,8.713659e+01
std,4.968873e-01,7.543183e-01,2.563828e-01,1.557189e+01,2.661676e+02,3.940147e-01,4.511853e-01,3.674926e-01,2.440971e-01,3.422669e-01,...,5.141761e+03,5.655884e+02,1.379283e+03,1.273109e+03,3.680647e+02,2.774419e+02,9.995214e+02,2.843982e+02,7.319756e+01,1.258293e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,-8.000000e+03,0.000000e+00,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+01
50%,2.000000e+00,1.000000e+00,2.000000e+00,2.500000e+01,3.300000e+02,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.400000e+02,1.500000e+02,0.000000e+00,8.800000e+01
max,2.000000e+00,5.000000e+00,2.000000e+00,5.400000e+01,9.990000e+02,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,...,1.566900e+05,3.730000e+04,8.300000e+04,4.051000e+04,1.104000e+04,1.800000e+04,1.653000e+04,3.590000e+03,3.000000e+03,1.150000e+02


Need to work on optimizing this step. It currently takes a little over 5 mins of run time to finish merging all 20 beneficiary files

In [55]:
start_time = time.time()

# Filter to include only the specific DESYNPUF_ID for EDA and QA
# merged_df = merged_df[merged_df['DESYNPUF_ID'] == '00016F745862898F']

# List of columns to sum across years
columns_to_sum = [
    'MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP',
    'MEDREIMB_OP', 'BENRES_OP', 'PPPYMT_OP',
    'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'
]

# Loop through each column and create a new column for the sum across the three years
for col in columns_to_sum:
    merged_df[col + '_SUM'] = merged_df[col + '_2008'] + merged_df[col + '_2009'] + merged_df[col + '_2010']

# Create a list of columns that end with '_SUM'
sum_columns = [col + '_SUM' for col in columns_to_sum]

# Add a new column 'total_sum' that sums all columns ending with '_SUM'
merged_df['total_sum'] = merged_df[sum_columns].sum(axis=1)

# Extract additional columns from the 2008 dataset to serve as our demographic features
additional_columns_08 = merged_df[['DESYNPUF_ID', 'BENE_BIRTH_DT_2008', 'Age', 'BENE_SEX_IDENT_CD_2008', 'BENE_RACE_CD_2008', 'SP_STATE_CODE_2008', 'BENE_COUNTY_CD_2008']]
# Rename columns to more human-readable names
additional_columns_08 = additional_columns_08.rename(columns={
    'BENE_BIRTH_DT_2008': 'Birth_Date',
    'BENE_SEX_IDENT_CD_2008': 'Gender',
    'BENE_RACE_CD_2008': 'Race',
    'SP_STATE_CODE_2008': 'State',
    'BENE_COUNTY_CD_2008': 'County'
})

# Export the wide format DataFrame to a CSV file for QA
# output_path_wide = r'C:\Users\JoeCarhart\OneDrive - Appriss Health LLC\Desktop\merged_data_wide.csv'
# merged_df.to_csv(output_path_wide, index=False)

# Reshape the DataFrame from wide to long format for the specified columns
columns_to_reshape = ['DESYNPUF_ID'] + [col + suffix for col in columns_to_sum for suffix in ['_2008', '_2009', '_2010']]
df_wide = merged_df[columns_to_reshape]

# Use pd.melt to reshape the DataFrame to long format
df_long = pd.melt(df_wide, id_vars=['DESYNPUF_ID'],
                  value_vars=[col + suffix for col in columns_to_sum for suffix in ['_2008', '_2009', '_2010']],
                  var_name='Year_Column', value_name='Value')

# Split the 'Year_Column' to extract the year information and the original column name
df_long['Year'] = df_long['Year_Column'].str.extract(r'(\d{4})$')
df_long['Column'] = df_long['Year_Column'].str.replace(r'_\d{4}$', '')

# Convert 'Year' to integer
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')

# Sum 'Value' by 'Year' and 'DESYNPUF_ID' to create 'annual_cost'
annual_cost = df_long.groupby(['DESYNPUF_ID', 'Year'])['Value'].sum().reset_index()
annual_cost.rename(columns={'Value': 'annual_cost'}, inplace=True)

# Finalize the long format DataFrame to include DESYNPUF_ID, Year, and annual_cost
df_long_final = annual_cost

# Merge additional columns from 2008 dataset
df_long_final = df_long_final.merge(additional_columns_08, on='DESYNPUF_ID', how='left')

# Export the long format DataFrame to a CSV file
output_path_long = r'C:\Users\JoeCarhart\OneDrive - Appriss Health LLC\Desktop\merged_data_long.csv'
df_long_final.to_csv(output_path_long, index=False)

# Display the first few rows of the resulting DataFrames
print("Wide format DataFrame:")
print(merged_df.head())
print("\nLong format DataFrame:")
print(df_long_final.head())

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

C:\Users\JoeCarhart\AppData\Local\Temp\ipykernel_18260\348429611.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  df_long['Column'] = df_long['Year_Column'].str.replace(r'_\d{4}$', '')


Wide format DataFrame:
        DESYNPUF_ID BENE_BIRTH_DT_2008  BENE_SEX_IDENT_CD_2008  \
0  00013D2EFD8E45D1         1923-05-01                       1   
1  00016F745862898F         1943-01-01                       1   
2  0001FDD721E223DC         1936-09-01                       2   
3  00021CA6FF03E670         1941-06-01                       1   
4  00024B3D2352D2D0         1936-08-01                       1   

   BENE_RACE_CD_2008  BENE_ESRD_IND_2008  SP_STATE_CODE_2008  \
0                  1                   2                  26   
1                  1                   2                  39   
2                  1                   2                  39   
3                  5                   2                   6   
4                  1                   2                  52   

   BENE_COUNTY_CD_2008  SP_ALZHDMTA_2008  SP_CHF_2008  SP_CHRNKIDN_2008  ...  \
0                  950                 2            2                 2  ...   
1                  230             

In [56]:
df_long_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6426861 entries, 0 to 6426860
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   DESYNPUF_ID  object        
 1   Year         int64         
 2   annual_cost  float64       
 3   Birth_Date   datetime64[ns]
 4   Age          int64         
 5   Gender       int64         
 6   Race         int64         
 7   State        int64         
 8   County       int64         
dtypes: datetime64[ns](1), float64(1), int64(6), object(1)
memory usage: 490.3+ MB


This code block needs further optimization. It currently takes close to 6 mins to process the diagnosis data for all 20 samples of beneficiary data

In [57]:
start_time = time.time()

# Define the diagnosis columns and prefixes
diagnosis_columns = [
    'DESYNPUF_ID',
    'SP_ALZHDMTA_2008', 'SP_CHF_2008', 'SP_CHRNKIDN_2008', 'SP_CNCR_2008', 'SP_COPD_2008',
    'SP_DEPRESSN_2008', 'SP_DIABETES_2008', 'SP_ISCHMCHT_2008', 'SP_OSTEOPRS_2008',
    'SP_RA_OA_2008', 'SP_STRKETIA_2008', 'BENE_ESRD_IND_2008',
    'SP_ALZHDMTA_2009', 'SP_CHF_2009', 'SP_CHRNKIDN_2009', 'SP_CNCR_2009', 'SP_COPD_2009',
    'SP_DEPRESSN_2009', 'SP_DIABETES_2009', 'SP_ISCHMCHT_2009', 'SP_OSTEOPRS_2009',
    'SP_RA_OA_2009', 'SP_STRKETIA_2009', 'BENE_ESRD_IND_2009',
    'SP_ALZHDMTA_2010', 'SP_CHF_2010', 'SP_CHRNKIDN_2010', 'SP_CNCR_2010', 'SP_COPD_2010',
    'SP_DEPRESSN_2010', 'SP_DIABETES_2010', 'SP_ISCHMCHT_2010', 'SP_OSTEOPRS_2010',
    'SP_RA_OA_2010', 'SP_STRKETIA_2010', 'BENE_ESRD_IND_2010'
]

# Create the diagnoses DataFrame from the main DataFrame
diagnoses = merged_df[diagnosis_columns]

# List of diagnosis prefixes
diagnosis_prefixes = [
    'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD',
    'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS',
    'SP_RA_OA', 'SP_STRKETIA', 'BENE_ESRD_IND'
]

# Initialize an empty DataFrame to store the long format data
diagnoses_long_list = []

# Transform each set of columns to long format and append them to the list
for prefix in diagnosis_prefixes:
    columns = [f'{prefix}_2008', f'{prefix}_2009', f'{prefix}_2010']
    df_long = pd.melt(diagnoses, id_vars=['DESYNPUF_ID'], value_vars=columns,
                      var_name='Year', value_name=prefix)
    df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})$').astype(int)
    diagnoses_long_list.append(df_long)

# Combine all long format DataFrames
diagnoses_long = diagnoses_long_list[0]
for df_long in diagnoses_long_list[1:]:
    diagnoses_long = diagnoses_long.merge(df_long, on=['DESYNPUF_ID', 'Year'], how='outer')

# Display the transformed long DataFrame
print("Long format Diagnoses DataFrame:")
print(diagnoses_long)

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

Long format Diagnoses DataFrame:
              DESYNPUF_ID  Year  SP_ALZHDMTA  SP_CHF  SP_CHRNKIDN  SP_CNCR  \
0        00013D2EFD8E45D1  2008            2       2            2        2   
1        00016F745862898F  2008            2       2            2        2   
2        0001FDD721E223DC  2008            2       2            2        2   
3        00021CA6FF03E670  2008            2       2            2        2   
4        00024B3D2352D2D0  2008            2       2            2        2   
...                   ...   ...          ...     ...          ...      ...   
6426856  FFFD961415966F79  2010            2       2            2        2   
6426857  FFFDA4E3D602964B  2010            2       2            2        2   
6426858  FFFEEB065C259516  2010            2       2            2        2   
6426859  FFFFAA8FF082E605  2010            2       1            2        2   
6426860  FFFFBC57C82B76C5  2010            2       1            1        2   

         SP_COPD  SP_DEPRESSN 

In [58]:
diagnoses_long.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6426861 entries, 0 to 6426860
Data columns (total 14 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   DESYNPUF_ID    object
 1   Year           int32 
 2   SP_ALZHDMTA    int64 
 3   SP_CHF         int64 
 4   SP_CHRNKIDN    int64 
 5   SP_CNCR        int64 
 6   SP_COPD        int64 
 7   SP_DEPRESSN    int64 
 8   SP_DIABETES    int64 
 9   SP_ISCHMCHT    int64 
 10  SP_OSTEOPRS    int64 
 11  SP_RA_OA       int64 
 12  SP_STRKETIA    int64 
 13  BENE_ESRD_IND  int32 
dtypes: int32(2), int64(11), object(1)
memory usage: 686.5+ MB


In [59]:
start_time = time.time()

# Merge diagnoses_long to df_long_final by 'DESYNPUF_ID' and 'Year'
bene_df = pd.merge(df_long_final, diagnoses_long, on=['DESYNPUF_ID', 'Year'], how='outer')

# Drop the 'Year' column
bene_df = bene_df.drop(columns=['Year'])

# Group by 'DESYNPUF_ID' and aggregate as required
agg_dict = {
    'Age': 'max',
    'Gender': 'max',
    'Race': 'max',
    'State': 'max',
    'County': 'max',
    'annual_cost': ['sum', 'mean'],
    'SP_ALZHDMTA': 'min',
    'SP_CHF': 'min',
    'SP_CHRNKIDN': 'min',
    'SP_CNCR': 'min',
    'SP_COPD': 'min',
    'SP_DEPRESSN': 'min',
    'SP_DIABETES': 'min',
    'SP_ISCHMCHT': 'min',
    'SP_OSTEOPRS': 'min',
    'SP_RA_OA': 'min',
    'SP_STRKETIA': 'min',
    'BENE_ESRD_IND': 'min'
}

# Perform the aggregation
bene_df = bene_df.groupby('DESYNPUF_ID').agg(agg_dict).reset_index()

# Flatten the MultiIndex columns
bene_df.columns = [col[0] if col[1] == '' else '_'.join(col).strip() for col in bene_df.columns]

# Remove suffixes '_max' and '_min' from column names
bene_df.columns = [col.replace('_max', '').replace('_min', '') for col in bene_df.columns]

# Rename the aggregated columns
bene_df = bene_df.rename(columns={
    'annual_cost_sum': 'total_cost',
    'annual_cost_mean': 'average_cost'
})

# Define the path to save the CSV file for QA
output_path = r'C:\Users\JoeCarhart\OneDrive - Appriss Health LLC\Desktop\final_merged_data.csv'

# Export the final DataFrame to a CSV file
bene_df.to_csv(output_path, index=False)

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

The code block took 55.96908164024353 seconds to run.


In [60]:
bene_df.head()

,DESYNPUF_ID,Age,Gender,Race,State,County,total_cost,average_cost,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,BENE_ESRD_IND
0,00000B48BCF4AD29,100,2,5,10,260,108092.0,36030.666667,1,1,1,1,2,1,1,1,1,2,1,1
1,0000141F2FECE9BC,50,2,1,39,560,5308.0,1769.333333,2,2,2,2,2,1,2,2,2,2,2,2
2,000022FFDB0BE2C7,85,1,1,24,260,290.0,96.666667,2,2,2,2,2,2,2,2,2,2,2,2
3,00002BE498BED936,83,1,1,36,480,0.0,0.000000,2,2,2,2,2,2,2,2,2,2,2,2
4,00003539A5D77654,80,1,1,24,690,290.0,96.666667,2,2,2,2,2,2,2,2,2,2,2,2


Inpatient data processing

In [61]:
start_time = time.time()

# Define base directory and file pattern
base_dir = r'C:\code_personal_use\cms\data\ip'
file_pattern = 'DE1_0_2008_to_2010_Inpatient_Claims_Sample_{}.csv'

# Function to load and preprocess a single file
def load_and_preprocess_file(file_path):
    df = pd.read_csv(file_path, usecols=['DESYNPUF_ID', 'CLM_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'AT_PHYSN_NPI', 'CLM_UTLZTN_DAY_CNT'], dtype={'AT_PHYSN_NPI': str})

    # Convert CLM_FROM_DT and CLM_THRU_DT to datetime format
    df['CLM_FROM_DT'] = pd.to_datetime(df['CLM_FROM_DT'], format='%Y%m%d', errors='coerce')
    df['CLM_THRU_DT'] = pd.to_datetime(df['CLM_THRU_DT'], format='%Y%m%d', errors='coerce')

    # Drop rows with missing values in CLM_FROM_DT, AT_PHYSN_NPI, and CLM_UTLZTN_DAY_CNT
    df = df.dropna(subset=['CLM_FROM_DT', 'AT_PHYSN_NPI', 'CLM_UTLZTN_DAY_CNT'])

    # Ensure AT_PHYSN_NPI does not have scientific notation and convert it to integers
    df['AT_PHYSN_NPI'] = df['AT_PHYSN_NPI'].apply(lambda x: int(float(x)))

    # Convert CLM_UTLZTN_DAY_CNT to integers
    df['CLM_UTLZTN_DAY_CNT'] = df['CLM_UTLZTN_DAY_CNT'].astype(int)

    # Extract the year as a four-digit number and create a new column 'Year'
    df['Year'] = df['CLM_FROM_DT'].dt.year

    return df

# List to store dataframes
dfs = []

# Load and preprocess all files
for i in range(1, 21):
    file_path = os.path.join(base_dir, file_pattern.format(i))
    df = load_and_preprocess_file(file_path)
    dfs.append(df)

# Concatenate all DataFrames
ip_sample_all = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the resulting DataFrame and unique ID counts
print(f'Unique IDs: {ip_sample_all["DESYNPUF_ID"].nunique()}')

print("First few rows of the DataFrame:")
print(ip_sample_all.head())

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

Unique IDs: 753231
First few rows of the DataFrame:
        DESYNPUF_ID           CLM_ID CLM_FROM_DT CLM_THRU_DT  AT_PHYSN_NPI  \
0  00013D2EFD8E45D1  196661176988405  2010-03-12  2010-03-13    3139083564   
1  00016F745862898F  196201177000368  2009-04-12  2009-04-18    6476809087   
2  00016F745862898F  196661177015632  2009-08-31  2009-09-02     611998537   
3  00016F745862898F  196091176981058  2009-09-17  2009-09-20    4971602784   
4  00016F745862898F  196261176983265  2010-06-26  2010-07-01    6408400473   

   CLM_UTLZTN_DAY_CNT  Year  
0                   1  2010  
1                   6  2009  
2                   2  2009  
3                   3  2009  
4                   5  2010  
The code block took 15.169937133789062 seconds to run.


In [62]:
ip_sample_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319470 entries, 0 to 1319469
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   DESYNPUF_ID         1319470 non-null  object        
 1   CLM_ID              1319470 non-null  int64         
 2   CLM_FROM_DT         1319470 non-null  datetime64[ns]
 3   CLM_THRU_DT         1319470 non-null  datetime64[ns]
 4   AT_PHYSN_NPI        1319470 non-null  int64         
 5   CLM_UTLZTN_DAY_CNT  1319470 non-null  int32         
 6   Year                1319470 non-null  int64         
dtypes: datetime64[ns](2), int32(1), int64(3), object(1)
memory usage: 65.4+ MB


In [63]:
start_time = time.time()

# Filter to include only the specific DESYNPUF_ID for EDA and QA
# ip_sample_one = ip_sample_one[ip_sample_one['DESYNPUF_ID'] == '00016F745862898F']

# Group by DESYNPUF_ID and Year, and perform the required calculations
ip_basic_summary = ip_sample_all.groupby(['DESYNPUF_ID']).agg({
    'CLM_ID': pd.Series.nunique,  # Count distinct CLM_ID
    'AT_PHYSN_NPI': pd.Series.nunique,  # Count distinct AT_PHYSN_NPI
    'CLM_UTLZTN_DAY_CNT': 'mean'  # Average CLM_UTLZTN_DAY_CNT
}).reset_index()

# Rename the columns to be more descriptive
ip_basic_summary.columns = ['DESYNPUF_ID', 'Distinct_IP_CLM_ID_Count', 'Distinct_IP_NPI_Count', 'Avg_CLM_UTLZTN_DAY_CNT']

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

The code block took 195.87233805656433 seconds to run.


In [64]:
ip_basic_summary

,DESYNPUF_ID,Distinct_IP_CLM_ID_Count,Distinct_IP_NPI_Count,Avg_CLM_UTLZTN_DAY_CNT
0,00000B48BCF4AD29,3,3,9.333333
1,0000141F2FECE9BC,1,1,2.000000
2,0000525AB30E4DEF,2,2,8.500000
3,000064755F16C901,1,1,6.000000
4,0000838E2BBC2ADA,2,1,14.000000
...,...,...,...,...
753226,FFFFBC57C82B76C5,2,2,2.000000
753227,FFFFC23B80C0069F,1,1,6.000000
753228,FFFFD21164D5316F,1,1,2.000000
753229,FFFFEE90E1B4D0DF,2,2,3.500000


Outpatient data processing

In [65]:
start_time = time.time()

# Define base directory and file pattern
base_dir = r'C:\code_personal_use\cms\data\op'
file_pattern = 'DE1_0_2008_to_2010_Outpatient_Claims_Sample_{}.csv'

# Function to load and preprocess a single file
def load_and_preprocess_file(file_path):
    df = pd.read_csv(file_path, usecols=['DESYNPUF_ID', 'CLM_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'AT_PHYSN_NPI'], dtype={'AT_PHYSN_NPI': str})

    # Convert CLM_FROM_DT and CLM_THRU_DT to datetime format
    df['CLM_FROM_DT'] = pd.to_datetime(df['CLM_FROM_DT'], format='%Y%m%d', errors='coerce')
    df['CLM_THRU_DT'] = pd.to_datetime(df['CLM_THRU_DT'], format='%Y%m%d', errors='coerce')

    # Drop rows with missing values in CLM_FROM_DT and AT_PHYSN_NPI
    df = df.dropna(subset=['CLM_FROM_DT', 'AT_PHYSN_NPI'])

    # Ensure AT_PHYSN_NPI does not have scientific notation and convert it to integers
    df['AT_PHYSN_NPI'] = df['AT_PHYSN_NPI'].apply(lambda x: int(float(x)))

    # Extract the year as a four-digit number and create a new column 'Year'
    df['Year'] = df['CLM_FROM_DT'].dt.year

    return df

# List to store dataframes
dfs = []

# Load and preprocess all files
for i in range(1, 21):
    file_path = os.path.join(base_dir, file_pattern.format(i))
    df = load_and_preprocess_file(file_path)
    dfs.append(df)

# Concatenate all DataFrames
op_sample_all = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the resulting DataFrame and unique ID counts
print(f'Unique IDs: {op_sample_all["DESYNPUF_ID"].nunique()}')

print("First few rows of the DataFrame:")
print(op_sample_all.head())

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

Unique IDs: 1702981
First few rows of the DataFrame:
        DESYNPUF_ID           CLM_ID CLM_FROM_DT CLM_THRU_DT  AT_PHYSN_NPI  \
0  00013D2EFD8E45D1  542192281063886  2008-09-04  2008-09-04    4824842417   
1  00016F745862898F  542272281166593  2009-06-02  2009-06-02    2963419753   
2  00016F745862898F  542282281644416  2009-06-23  2009-06-23    5737807753   
3  0001FDD721E223DC  542642281250669  2009-10-11  2009-10-11    1233847710   
4  00024B3D2352D2D0  542242281386963  2008-07-12  2008-07-12    9688809345   

   Year  
0  2008  
1  2009  
2  2009  
3  2009  
4  2008  
The code block took 142.4021863937378 seconds to run.


In [66]:
op_sample_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15474093 entries, 0 to 15474092
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   DESYNPUF_ID   object        
 1   CLM_ID        int64         
 2   CLM_FROM_DT   datetime64[ns]
 3   CLM_THRU_DT   datetime64[ns]
 4   AT_PHYSN_NPI  int64         
 5   Year          int64         
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 708.3+ MB


In [ ]:
start_time = time.time()

# Filter to include only the specific DESYNPUF_ID for EDA and QA
# op_sample_one = op_sample_one[op_sample_one['DESYNPUF_ID'] == '00016F745862898F']

# Group by DESYNPUF_ID and Year, and perform the required calculations
op_basic_summary = op_sample_all.groupby(['DESYNPUF_ID']).agg({
    'CLM_ID': pd.Series.nunique,  # Count distinct CLM_ID
    'AT_PHYSN_NPI': pd.Series.nunique,  # Count distinct AT_PHYSN_NPI
}).reset_index()

# Rename the columns to be more descriptive
op_basic_summary.columns = ['DESYNPUF_ID', 'Distinct_OP_CLM_ID_Count', 'Distinct_OP_NPI_Count']

stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

In [ ]:
op_basic_summary

A few quick checks before combining the benecificary data, inpatient data, and outpatient data

In [ ]:
bene_df.info()

In [ ]:
ip_basic_summary.info()

In [ ]:
op_basic_summary.info()

Merge the three data sets

In [ ]:
start_time = time.time()

# Select the specified columns from ip_basic_summary and op_basic_summary
ip_selected = ip_basic_summary[['DESYNPUF_ID', 'Distinct_IP_CLM_ID_Count', 'Distinct_IP_NPI_Count', 'Avg_CLM_UTLZTN_DAY_CNT']]
op_selected = op_basic_summary[['DESYNPUF_ID', 'Distinct_OP_CLM_ID_Count', 'Distinct_OP_NPI_Count']]


# Merge bene and ip data
bene_ip = pd.merge(bene_df, ip_selected, on=['DESYNPUF_ID'], how='left')


# Merge the result with outpatient data
final_df = pd.merge(bene_ip, op_selected, on=['DESYNPUF_ID'], how='left')


# Replace NaNs with 0
final_df.fillna(0, inplace=True)


stop_time = time.time()
print("The code block took {0} seconds to run.".format(stop_time - start_time))

In [ ]:
final_df

**Perform predictive modeling**

I have chosen to use the XGBoost regression model for this particular predictive project.


XGBoost (Extreme Gradient Boosting) is a powerful machine learning algorithm known for its performance and efficiency. Below are several benefits of using XGBoost for regression tasks compared to other methods:

Performance and Accuracy
* XGBoost often outperforms other algorithms in terms of predictive accuracy. It efficiently handles large datasets and can model complex relationships and can handle missing data gracefully by learning the best direction to handle missing values in the decision trees.

Regularization
* XGBoost incorporates both L1 (Lasso) and L2 (Ridge) regularization, which helps prevent overfitting and improves model generalization.

Flexibility
* XGBoost allows for the implementation of custom loss functions, providing flexibility to tailor the model to specific problem requirements and offers a wide range of parameters that can be tuned to improve model performance, such as learning rate, tree depth, and subsampling ratios.

Efficiency and Scalability
* XGBoost supports parallel processing, making it much faster than other gradient boosting implementations and can be run on distributed systems, enabling the processing of very large datasets that do not fit into memory.

Tree Pruning and Sparsity Aware
* XGBoost uses a more sophisticated tree pruning algorithm that reduces overfitting and handles sparse data efficiently, which is particularly beneficial for datasets with missing values or categorical features that have been one-hot encoded.

Cross-Validation
* Built-in Cross-Validation: XGBoost has built-in cross-validation capabilities, making it easier to assess model performance and prevent overfitting.

Feature Importance
* Feature Importance Analysis: XGBoost provides useful metrics for feature importance, which helps in understanding the underlying data and the factors influencing the predictions.

Comparison with Other Methods
* Linear Regression: XGBoost can capture non-linear relationships which linear regression cannot.
* Random Forests: XGBoost often provides better performance and can handle bias-variance trade-off more effectively due to its boosting nature.
* Neural Networks: While neural networks can also model complex relationships, they typically require more computational resources and longer training times. XGBoost tends to be faster and easier to tune.
* Other Boosting Methods (e.g., AdaBoost): XGBoost typically offers better performance due to its regularization techniques and advanced handling of missing data and sparsity.

In [ ]:
# Let's convert to traditional dummy coded values for easier end-user interpretation.
# Columns to transform
columns_to_transform = ['SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD',
                        'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS',
                        'SP_RA_OA', 'SP_STRKETIA', 'BENE_ESRD_IND']

# Transform the values
final_df[columns_to_transform] = final_df[columns_to_transform].replace({2: 0, 1: 1})

In [ ]:
final_df

In [ ]:
# Split data into training and validation sets
train, test = train_test_split(final_df, test_size=0.2, random_state=42)

In [ ]:
train.shape, test.shape

In [ ]:
train.columns.tolist()

In [ ]:
X_train = train[[ 'Age',
 'Gender',
 'Race',
 'State',
 'County',
 'SP_ALZHDMTA',
 'SP_CHF',
 'SP_CHRNKIDN',
 'SP_CNCR',
 'SP_COPD',
 'SP_DEPRESSN',
 'SP_DIABETES',
 'SP_ISCHMCHT',
 'SP_OSTEOPRS',
 'SP_RA_OA',
 'SP_STRKETIA',
 'BENE_ESRD_IND',
 'Distinct_IP_CLM_ID_Count',
 'Distinct_IP_NPI_Count',
 'Avg_CLM_UTLZTN_DAY_CNT',
 'Distinct_OP_CLM_ID_Count',
 'Distinct_OP_NPI_Count',
 'average_cost',]]
y_train = train[['total_cost']]

In [ ]:
X_test = test[[ 'Age',
 'Gender',
 'Race',
 'State',
 'County',
 'SP_ALZHDMTA',
 'SP_CHF',
 'SP_CHRNKIDN',
 'SP_CNCR',
 'SP_COPD',
 'SP_DEPRESSN',
 'SP_DIABETES',
 'SP_ISCHMCHT',
 'SP_OSTEOPRS',
 'SP_RA_OA',
 'SP_STRKETIA',
 'BENE_ESRD_IND',
 'Distinct_IP_CLM_ID_Count',
 'Distinct_IP_NPI_Count',
 'Avg_CLM_UTLZTN_DAY_CNT',
 'Distinct_OP_CLM_ID_Count',
 'Distinct_OP_NPI_Count',
 'average_cost',]]
y_test = test[['total_cost']]

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
# Define the XGBoost model
model = XGBRegressor(
    objective='reg:squarederror',  # Specify regression task
    eval_metric='rmse',            # Root Mean Squared Error
    max_depth=3,                   # Maximum tree depth
    eta=0.1,                       # Learning rate
    subsample=0.8,                 # Subsample ratio of training instances
    colsample_bytree=0.8,          # Subsample ratio of columns when constructing each tree
    n_estimators=100,              # Number of boosting rounds
    # early_stopping_rounds=10       # Early stopping rounds
)

In [ ]:
# Define the  outcome and covariates for cross-validation
# Separate the features and the target
X = final_df.drop(columns=['total_cost', 'DESYNPUF_ID'])
y = final_df['total_cost']

In [ ]:
X

In [ ]:
y

In [ ]:
# Perform K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')

# Convert negative MSE to positive
cv_results = -cv_results

print(f'Cross-Validation MSE: {cv_results}')
print(f'Mean Cross-Validation MSE: {np.mean(cv_results)}')
print(f'Standard deviation of Cross-Validation Score: {np.std(cv_results)}')

# Plot cross-validation results
plt.figure(figsize=(8, 6), facecolor='white')  # Set the figure background color to white
ax = plt.gca()
ax.set_facecolor('white')  # Set the axes background color to white

# Create the boxplot
plt.boxplot(cv_results, vert=False)
plt.xlabel('Mean Squared Error')
plt.title('Cross-Validation Results')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)  # Optional: Add grid lines for better readability
plt.show()

**The basics:**

Performance Consistency: The boxplot helps visualize the consistency of our model's performance across different folds of the cross-validation. A tight box and short whiskers indicate consistent performance, whereas a wide box and long whiskers indicate more variability.

Central Tendency and Spread: The median and IQR provide insight into the central tendency and spread of the cross-validation results.

Potential Issues: Outliers highlight potential issues or variations in performance that might need further investigation. They can be caused by particular subsets of the data that are more challenging to predict.

**Interpretation:**

Range of MSE Values: The MSE values range approximately from 3.4e6 to 4.2e6.

Median MSE: The median MSE value appears to be close to the center of the IQR box, indicating a relatively symmetrical distribution of errors within the central 50% of the data.

Outliers: The presence of outliers suggests that in some folds, the model performed significantly better or worse compared to the majority of the folds.

In [ ]:
model.fit(X_train, y_train.values.ravel())

In [ ]:
# Predict on the validation set
y_pred = model.predict(X_test)

In [ ]:
test.loc[:,'predicted_2011_cost'] = y_pred.tolist()

In [ ]:
X_test.info()

In [ ]:
# Calculate performance metrics on the test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R2: {r2}')

Mean Absolute Error (MAE): Average absolute difference between predicted and actual values.

Mean Squared Error (MSE): Lower values indicate better performance.

Root Mean Squared Error (RMSE): Similar to MSE but in the same units as the target variable. So, the model performs pretty well since we are trying to predict total cost and the units here are dollars ($1,860.11)

R-squared (R²): Proportion of variance explained by the model (higher is better).

In [ ]:
# feature selection
def select_features(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=f_classif, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train.values.ravel())
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)

# get feature scores and names
feature_scores = fs.scores_
feature_names = X_train.columns

# combine names and scores into a list of tuples
features_and_scores = list(zip(feature_names, feature_scores))

# sort the list of tuples in descending order of scores
features_and_scores.sort(key=lambda x: x[1], reverse=True)

# print the sorted feature scores with their names
for name, score in features_and_scores:
    print(f'Feature {name}: {score}')

# plot the sorted scores
sorted_names = [x[0] for x in features_and_scores]
sorted_scores = [x[1] for x in features_and_scores]

plt.figure(figsize=(12, 10))  # Increase the figure size
plt.bar(sorted_names, sorted_scores)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Create figure
# fig, ax = plt.subplots(figsize=(12, 10))  # Adjust the figsize as needed
# plot_importance(model, ax=ax)
# plt.show()

In [ ]:
# Calculate the threshold for the top 10% of predictions
threshold = np.percentile(test['predicted_2011_cost'], 90)  # Top 10%

# Identify high-risk patients with predictions in the top 10%
high_risk_patients = test[test['predicted_2011_cost'] >= threshold]

# Extract DESYNPUF_ID and prediction for high-risk patients
high_risk_patients_list = high_risk_patients[['DESYNPUF_ID', 'predicted_2011_cost']]

# Define the file path for the output CSV file
file_path = r'C:\Users\JoeCarhart\OneDrive - Appriss Health LLC\Desktop\high_risk_patients.csv'

# Save the DataFrame to a CSV file
high_risk_patients_list.to_csv(file_path, index=False)

print(f"High-risk patients list saved to {file_path}")

In [ ]:
high_risk_patients_list